<a href="https://colab.research.google.com/github/Danzhy/CheapestFlightsLTD./blob/main/CheapestFlights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up authentication to work with Amadeus API

Our team chose to work with the [Amadeus API](https://developers.amadeus.com/). This API allows developers to fetch data regarding plane tickets. There are many search types and filters that can be applied on plane ticket search like duration of stay, departure date, maximum price, etc. I chose a search that requires the user to input their origin city/airport so that API can show cheapest flights from that location.

This application works with two APIs - Amadeus API for plane tickets and Currencybeacon API for currency exchange.

Amadeus API requires developers to obtain access token by making a POST request for the purpose of using their API functionality.

The POST request must contain:

*   API key
*   API secret


The following code snippet is designed to obtain access token and save it as a global variable "access_token"

In [35]:
from google.colab import userdata
import requests

#According to documentation, this url is used to get access token.
#Access token is necessary to work with this flight tickets api.
token_url = "https://test.api.amadeus.com/v1/security/oauth2/token"

#Getting api key and api secret
#This key and this secret are from the Amadeus API to fetch plane ticket data
key_flight = userdata.get('FLIGHT')
secret_flight = userdata.get('FLIGHT_SECRET')

#necessary to include header
headers_flight = {
    "Content-Type": "application/x-www-form-urlencoded"
}

#necessary to include information to get access token
data = {
    "grant_type": "client_credentials",
    "client_id": key_flight,
    "client_secret": secret_flight
}

#saving access token in the global variable "access_token"
response = requests.post(token_url, headers = headers_flight, data = data)
token_response = response.json()
access_token = token_response['access_token']

#Functions for API requests

**Function get_flight(origin_city)**

*   ListIt makes an API request to the Amadeus API. It takes an IATA code as variable and incorporates it in the API request url.
*   In the header of the request we put access token.


*   This function, in case of successful request, returns a json file with file tickets information



**Function exchange_currency(ororig_curr, dest_curr = 'AED', amount = '100')**



*   It makes an API request to the Currencybeacon API. It takes two currency types and amount as variables
*   In case of successful request, function returns a json file that contains the new amount based on exchange rate





In [31]:
import requests

def get_flight(origin_city):
  header_auth = "Bearer " + access_token #global access_token variable is appended to Bearer for authentication

  #Url of the request, where origin city/airport depends on user input
  base_url = f"https://test.api.amadeus.com/v1/shopping/flight-destinations?origin={origin_city}"
  headers_get = {
      "Authorization": header_auth
  }

  get_response = requests.get(base_url, headers = headers_get)

  #To avoid errors .json() method is applied only in case of successful request and then returned
  if get_response.status_code == 200:
    return get_response.json()
  else:
    return -1

#Default values are needed for cheking validity of original currency
def exchange_currency(orig_curr, dest_curr = 'AED', amount = '100'):

  #API key for currencybeacon API
  key_exchange = userdata.get('CURRENCY')

  url_curr = "https://api.currencybeacon.com/v1/convert"

  params_curr = {
      "api_key": key_exchange,
      "from": orig_curr,
      "to": dest_curr,
      "amount": amount
  }

  get_response = requests.get(url_curr, params = params_curr)

  #To avoid errors .json() method is applied only in case of successful request and then returned
  if get_response.status_code == 200:
    return get_response.json()
  else:
    return -1




#Execution of the application

In this code snippet is responsible for interactions with user, and calling API request functions.

User is promted to select desired currency and origin city (city a they want to fly from). Based on this information, the programm will output 5 cheapest plane tickets from this origin_city to any other destination.

In [34]:
import json

print("Hi! This program helps you find cheapest flights originating in your city!")

#The program initially asks user to input currency type (USD, EUR, etc.) that user is comfortable with
#The while loop is needed to check if the entered currency exists/written correctly
check = True
global currency
while(check):
  currency = input("First of all, please enter currency that you want us to use (USD, EUR, AED, CNY, etc.)\n")

  if (exchange_currency(currency) == -1):
    print("There is no such currency! Please enter a valid currency!")

  else:
    check = False

#This input prompts the user to enter the IATA code to make an API call
origin = input("Please enter the IATA code of your origin city or airport.\n" "For example, LON is code for London, and ICN is a code for Incheon airport.\n")

#In this variable the json file with plane tickets is saved
flight_list = get_flight(origin)


#With if statement we check if the request was successful. Only in case of success we print the plane tickets.
#This prevents the error of trying to access undefined data with flight_list['data'][:5]
if(flight_list != -1):
# Loop over the first 5 flight destinations and print their details
  for i, ticket in enumerate(flight_list['data'][:5]):
    exchanged = exchange_currency('EUR', currency, ticket['price']['total'])
    newPrice = exchanged['response']['value']

    print(f"Ticket {i + 1}:")
    print(f"Origin: {ticket['origin']}")
    print(f"Destination: {ticket['destination']}")
    print(f"Departure Date: {ticket['departureDate']}")
    print(f"Return Date: {ticket['returnDate']}")
    print(f"Price: {round(newPrice, 2)} {currency}")
    print()

else:
  print("Sorry, you entered incorrect city/airport code!")

Hi! This program helps you find cheapest flights originating in your city!
First of all, please enter currency that you want us to use (USD, EUR, AED, CNY, etc.)
XCV
There is no such currency! Please enter a valid currency!
First of all, please enter currency that you want us to use (USD, EUR, AED, CNY, etc.)
13
There is no such currency! Please enter a valid currency!
First of all, please enter currency that you want us to use (USD, EUR, AED, CNY, etc.)
AED
Please enter the IATA code of your city or airport.
For example, LON is code for London, and ICN is a code for Incheon airport.
AFDXC
Sorry, you entered incorrect city/airport code!
